In [1]:
from sklearn.svm import SVC
import torchvision.transforms as tt
import torch.nn as nn 
from utils import *
import config

In [2]:
print_config()

RANDOM_SEED   :  11042004
DATA_DIR      :    ./data
USED_DATA     :    DOODLE
NUM_LABELLED  :       500
DEVICE        :    cuda:0
EPOCHS        :        20
BATCH_SIZE    :        32
LEARNING_RATE :    0.0002
SCHED         :     False
GAN_BATCH_SIZE:       128


In [3]:
set_random_seed(config.RANDOM_SEED)

Setting seeds ...... 



In [4]:
name = 'SVM'

In [5]:
PATH = get_PATH(name)
PATH

'DOODLE/SVM/_500'

In [6]:
if config.USED_DATA == "CIFAR10":
	mean = [0.5]*3
	std = [0.5]*3

	train_tfm = tt.Compose([
		tt.Resize(32),
		tt.RandomCrop(32, padding=4, padding_mode='edge'),
		tt.RandomHorizontalFlip(),
		tt.Normalize(mean, std, inplace=True)
	])

if config.USED_DATA == "MNIST" or config.USED_DATA == "DOODLE":
	mean = [0.5]
	std = [0.5]
	train_tfm = tt.Compose([
		tt.Resize(32),
		tt.Normalize(mean, std, inplace=True)
	])

test_tfm = tt.Compose([
	tt.Resize(32),
	tt.Normalize(mean, std)
])

In [7]:
train_ds, test_ds, classes = load_data(train_tfm, test_tfm)

In [8]:
n_classes = len(classes)
channels = train_ds[0][0].shape[0] # MNIST
n_classes, channels

(10, 1)

In [9]:
X_full = train_ds.x 
y_full = train_ds.y

In [10]:
flatten = nn.Flatten()

In [11]:
X_sup, y_sup, X_unsup, _ = supervised_samples(X_full, y_full, config.NUM_LABELLED, n_classes, get_unsup=True)

In [12]:
X_sup = flatten(X_sup) 

In [13]:
X_unsup = flatten(X_unsup)

In [14]:
X_test = test_ds.x
y_test = test_ds.y
X_test = flatten(X_test)

In [15]:
print(X_sup.shape)
print(y_sup.shape)

torch.Size([500, 784])
torch.Size([500])


In [16]:
model = SVC(probability=True)
model.fit(X_sup, y_sup)

SVC(probability=True)

In [17]:
model.score(X_test, y_test)

0.681

In [18]:
y_unsup = model.predict(X_unsup)
y_unsup

array([0, 0, 0, ..., 9, 2, 9])

In [16]:
import joblib

In [ ]:
joblib.dump(model, PATH + ".pkl")

In [17]:
model1 = joblib.load(PATH + ".pkl")

In [18]:
model1.score(X_test, y_test)

0.667